In [6]:
;;; A very small knowledge base for diagnosing infections
(define rules
  '(
    (rule-1 ; Rule Name
     (treat-with penicillin) ; Conclusion
     0.8 ; Certainty Factor
     ((gram-positive ?organism) ; Premise (list of conditions)
      (shape rod ?organism)))

    (rule-2
     (gram-positive e-coli)
     -0.9 ; Strong evidence against
     ((stains pink ?organism)))

    (rule-3
     (gram-positive staphylococcus)
     0.9
     ((stains purple ?organism)
      (shape coccus ?organism)))

    (rule-4 ; A rule that asks the user for data
     (shape ?shape ?organism)
     1.0 ; User input is taken as certain
     ((ask-shape ?organism ?shape))) ; Special predicate 'ask-shape'

    (rule-5
     (stains ?stain ?organism)
     1.0
     ((ask-stain ?organism ?stain)))
  ))

In [7]:
(define working-memory '())

;; Function to add a new fact to working memory
(define (add-fact fact cf)
  (let ((existing (assoc fact working-memory))) ; Check if we already know this
    (if existing
        (set-cdr! existing (combine-cf (cdr existing) cf)) ; Combine certainty
        (set! working-memory (cons (cons fact cf) working-memory))))) ; Add new fact

;; A very simple certainty combination function (MYCIN's was more complex)
(define (combine-cf cf-a cf-b)
  (if (and (> cf-a 0) (> cf-b 0))
      (+ cf-a (* cf-b (- 1 cf-a))) ; Both positive
      (* cf-a cf-b))) ; Other cases simplified for example

In [8]:
;; The main inference function: tries to prove a goal fact
(define (check-fact goal)
  (display (string-append "Trying to prove: " (->string goal) "\n"))
  ;; First, check if it's already in working memory
  (let ((known (assoc goal working-memory)))
    (if known
        (begin
          (display (string-append "  Already known with CF: " (number->string (cdr known)) "\n"))
          (cdr known)) ; Return its certainty

        ;; If not known, try to find a rule that concludes it
        (let loop ((rules-to-try rules))
          (if (null? rules-to-try)
              0.0 ; No rules found, CF=0 (unknown)
              (let* ((rule (car rules-to-try))
                     (conclusion (cadr rule))
                     (rule-cf (caddr rule))
                     (premise (cadddr rule)))
                ;; Check if this rule's conclusion UNIFIES with our goal
                (if (unify conclusion goal) ; unification binds variables
                    (let ((bindings (unify conclusion goal)))
                      (display (string-append "  Found rule: " (->string (car rule)) "\n"))
                      ;; Instantiate the premise with the bindings from unification
                      (let ((instantiated-premise (instantiate premise bindings)))
                        ;; Try to prove ALL conditions in the premise
                        (let ((premise-cf (check-premise instantiated-premise)))
                          (if (> premise-cf 0)
                              (let ((final-cf (* premise-cf rule-cf))) ; Combine rule CF with premise CF
                                (add-fact (instantiate (list goal) bindings) final-cf) ; Add to WM
                                (display (string-append "  Concluded: " (->string goal) " with CF: " (number->string final-cf) "\n"))
                                final-cf)
                              (loop (cdr rules-to-try)))))) ; Premise failed, try next rule
                    (loop (cdr rules-to-try)))))))) ; Conclusion didn't match, try next rule
)

;; Function to check a list of premises (AND them together)
(define (check-premise premises)
  (if (null? premises)
      1.0 ; Empty premise is always true (CF=1)
      (let ((cf (check-fact (car premises)))) ; Check the first condition
        (if (<= cf 0)
            0.0 ; If any condition is false, the whole AND is false
            (* cf (check-premise (cdr premises))))))) ; Combine with the rest

In [9]:
;;; --- Simplified Unification & Support Functions ---
(define (variable? x)
  (and (symbol? x) (char=? (string-ref (symbol->string x) 0) #\?)))

;; A very basic unifier for this example
(define (unify pattern fact . bindings)
  (cond ((null? bindings) #f)
        ((null? pattern) (if (null? fact) (car bindings) #f))
        ((variable? (car pattern))
         (let ((binding (assoc (car pattern) (car bindings))))
           (if binding
               (if (equal? (cdr binding) (car fact))
                   (unify (cdr pattern) (cdr fact) (car bindings))
                   #f)
               (unify (cdr pattern) (cdr fact) (cons (cons (car pattern) (car fact)) (car bindings))))))
        ((equal? (car pattern) (car fact))
         (unify (cdr pattern) (cdr fact) (car bindings)))
        (else #f)))

(define (instantiate pattern bindings)
  (map (lambda (term)
         (if (variable? term)
             (let ((value (assoc term bindings)))
               (if value (cdr value) term))
             term))
       pattern))

;;; --- Interaction with the User ---
(define (ask-shape organism shape)
  (display (string-append "What is the shape of " (->string organism) "? (rod/coccus)\n> "))
  (let ((answer (read)))
    (if (equal? answer shape)
        1.0
        0.0)))

(define (ask-stain organism stain)
  (display (string-append "What is the Gram stain of " (->string organism) "? (purple/pink)\n> "))
  (let ((answer (read)))
    (if (equal? answer stain)
        1.0
        0.0)))

(define (->string obj)
  (format #f "~a" obj))

In [10]:
;;; --- Run the system ---
(define (run-mycin)
  (set! working-memory '()) ; Clear previous facts
  (check-fact '(treat-with penicillin))) ; Set the top-level goal

(run-mycin)

Trying to prove: (treat-with penicillin)
0.0